In [52]:
import sys
import pysam
import truvari
import json
import pandas as pd
import joblib
import numpy as np

SIZEMIN = 50
SIZEMAX=10000
NEIGHLIMIT = 6
program='test'
technology='test'
# This should have numneigh annotation inside
base_vcf = "dipcall_bcftools_merge_neigh.vcf.gz"
bed_fn = "dipcall.bed"
comp_vcf = "sniffles_fixed.vcf.gz"
bench_jl = "bench_sniffles/data.jl"
bench_summary = "bench_sniffles/summary.json"

###

# Load the data
data = joblib.load(bench_jl)

# Separate out the variants from the base VCF and add new columns of the base/comp ids
base = data[data['state'].isin(['tpbase', 'fn'])].copy()
base['base_id'] = base['MatchId'].apply(lambda x: x[0])
base['comp_id'] = base['MatchId'].apply(lambda x: x[1])

# Separate out the variants from the comparison VCF and add new columns of the base/comp ids
comp = data[data['state'].isin(['tp', 'fp'])].copy()
comp['base_id'] = comp['MatchId'].apply(lambda x: x[0])
comp['comp_id'] = comp['MatchId'].apply(lambda x: x[1])

# Merge the base/comparison variants
combined = pd.merge(base, comp, left_on='base_id', right_on='comp_id', suffixes=('_base', '_comp'))

# How many comp variants matched to multiple base variants?
counts1 = combined['base_id_comp'].value_counts()
print('multi-matched comp count', (counts1 != 1).sum())

# How many base variants matched to multiple comp variants?
counts2 = combined['comp_id_base'].value_counts()
print('multi-matched base count', (counts2 != 1).sum())

# For every tp-base, I can check its GTMatch to figure out concordance/discordance
#.. but that doesn't solve the ALstate lookups..
# So I need to use a vcf2df 
# and for every fn, I add that to discorant
# and for every fp, I add that to discordant
# and then I can add to concordant every reference homozygous base_vcf
# And then I can add to missing every Missing


multi-matched comp count 3020
multi-matched base count 259


In [36]:
m_cnt_base = {'Concordant': 0,
          'Discordant': 0,
          'Missing': 0,
          'REFHOM': 0}

m_cnt = {'DEL': dict(m_cnt_base),
         'INS': dict(m_cnt_base),
         'TOT': dict(m_cnt_base)}
m_neigh_cnt = {}
for i in range(0, NEIGHLIMIT):
    m_neigh_cnt[i] = dict(m_cnt_base)

# These are relative to the comparison VCF
matched = (comp['state'] == 'tp') & (comp['GTMatch'] == 0)
m_cnt['TOT']['Concordant'] = matched.sum()
m_cnt['TOT']['Discordant'] = (~matched).sum() + (base['state'] == 'fn').sum()

sub = comp[comp['svtype'] == 'DEL']
matched_del = (sub['state'] == 'tp') & (sub['GTMatch'] == 0)
m_cnt['DEL']['Concordant'] = matched_del.sum()
m_cnt['DEL']['Discordant'] = (~matched_del).sum() \
                            + ((base['svtype'] == 'DEL') & (base['state'] == 'fp')).sum()

sub = comp[comp['svtype'] == 'INS']
matched_del = (sub['state'] == 'tp') & (sub['GTMatch'] == 0)
m_cnt['INS']['Concordant'] = matched_del.sum()
m_cnt['INS']['Discordant'] = (~matched_del).sum() \
                            + ((base['svtype'] == 'INS') & (base['state'] == 'fp')).sum()


vcf = pysam.VariantFile(comp_vcf)
bed = truvari.build_region_tree(vcfA=vcf, includebed=bed_fn)
if 'chrX' in bed:
    del(bed['chrX'])
if 'chrY' in bed:
    del(bed['chrY'])
for i in list(bed.keys()):
    if '_' in i:
        del(bed[i])

vcf_i = truvari.region_filter(vcf, bed)

for entry in vcf_i:
    sz = truvari.entry_size(entry)
    if sz < SIZEMIN or sz > SIZEMAX:
        continue
    m_gt = entry.samples[0]['GT']
    svtype = truvari.entry_variant_type(entry).name
    # Already accounted for
    if 1 in m_gt:
        continue
    if None in m_gt:
        m_cnt['TOT']['Missing'] += 1
        m_cnt[svtype]['Missing'] += 1 
    else:
        m_cnt['TOT']['REFHOM'] += 1
        m_cnt[svtype]['REFHOM'] += 1

In [57]:
parts = []
for k,v in m_cnt.items():
    d = pd.Series(v)
    d['svtype'] = k
    parts.append(d.to_frame())
summary = pd.concat(parts, axis=1)
summary = summary.T.reset_index(drop=True)
summary['GT Concordance'] = summary['Concordant'] / (summary['Concordant'] + summary['Discordant'])


base['NeighBin'] = pd.cut(base['NumNeighbors'], bins=[0, 1, 2, 3, 4, 5, sys.maxsize],
                            labels=['0', '1', '2', '3', '4', '>=5'])
       
base['CorrectGT'] = (base['state'] == 'tpbase') & (base['GTMatch'] == 0)

neigh_sum = base.groupby(['NeighBin', 'CorrectGT']).size().unstack()
neigh_sum['pct'] = neigh_sum[True] / neigh_sum.sum(axis=1)
neigh_sum['total'] = neigh_sum[True] + neigh_sum[False]
neigh_sum['pct of baseline']  = neigh_sum['total'] / neigh_sum['total'].sum()

perf_sum = json.load(open(bench_summary))
del(perf_sum['gt_matrix'])
perf_sum['Baseline GT Concordance'] = perf_sum['gt_concordance']
del(perf_sum['gt_concordance'])
perf_sum['Concordant'] = perf_sum['TP-base_TP-gt']
perf_sum['GT Accuracy'] = perf_sum['TP-base_TP-gt'] / perf_sum['base cnt']
del(perf_sum['TP-comp_TP-gt'])
del(perf_sum['TP-comp_FP-gt'])
del(perf_sum['TP-base_TP-gt'])
del(perf_sum['TP-base_FP-gt'])

perf_sum = pd.DataFrame([perf_sum])


/var/folders/fy/lkd5_yz90zq3f67qys3tpztc0000gn/T/ipykernel_73335/205914601.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  neigh_sum = base.groupby(['NeighBin', 'CorrectGT']).size().unstack()


In [54]:
gtc = comp['GT'].apply(truvari.get_gt).value_counts()

m_gtarray = np.zeros(4)
m_gtarray[0] = m_cnt['TOT']["REFHOM"]
m_gtarray[1] = gtc[truvari.GT.HET]
m_gtarray[2] = gtc[truvari.GT.HOM]
m_gtarray[3] = m_cnt['TOT']['Missing']
m_gtarray

array([168049.,  97753., 103178.,   2054.])

In [58]:
# Matching the non-reference-homozygous variants between the baseline multi-sample VCF
# and the genotyper results
# Recall is how many baseline variants are still present in the genotyper results
# Precision is how many comparison variants are present and have ≥90% similarity to a present baseline variant
# baseline gt_concordance is the percent of baseline variants with a correct genotype
perf_sum

,TP-base,TP-comp,FP,FN,precision,recall,f1,base cnt,comp cnt,baseline gt_concordance,Concordant,GT Concordant
0,23502,23510,177421,1987,0.117005,0.922045,0.207659,25489,200931,0.71829,16886,0.662482


In [41]:
# For the comparison variants, we can count how many genotypes are concordant as those are present
# And match to a present baseline variant. We can also count the number of reference homozygous variants
summary

,Concordant,Discordant,Missing,REFHOM,svtype,GT Concordance
0,7529,31879,460,50298,DEL,0.191053
1,9358,152165,1594,117751,INS,0.057936
2,16887,186031,2054,168049,TOT,0.083221


In [46]:
# For the baseline variants, we count what percent of the present variants are by number of neighbors
neigh_sum.round(4)

CorrectGT,False,True,pct,total,pct of baseline
NeighBin,,,,,
0,171,1175,0.8730,1346,0.0628
1,210,1097,0.8393,1307,0.0610
2,223,831,0.7884,1054,0.0492
3,253,669,0.7256,922,0.0430
4,210,575,0.7325,785,0.0366
>=5,7355,8660,0.5407,16015,0.7474
